In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when
from pyspark.sql.functions import row_number, desc
import pyspark.sql.functions as sf

from datetime import datetime
import os

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()

23/11/07 22:13:00 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
23/11/07 22:13:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 22:13:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/11/07 22:13:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
def etl_process(path, file_name, file_date):
    spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()
    
    
    df = spark.read.json(path+file_name)
    df = df.select('_source.*')
    
    df = df.withColumn("Type",
           when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    
    df = df.withColumn("Date", sf.lit(file_date))
    df = df.select('Contract','Type','TotalDuration', 'Date')
    df = df.filter(df.Type != 'Error')
    
    print('Finished Processing {}'.format(file_name))
    return df

In [4]:
path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'
list_file = sorted([file for file in os.listdir(path) if file != '.DS_Store'])

In [5]:
start_date_str = '20220401'
end_date_str = '20220430'

In [6]:
file_name = "20220401.json"
date_str = file_name.split('_')[-1].split('.')[0]
file_date = datetime.strptime(date_str, "%Y%m%d").date()

In [7]:
test = etl_process(path, file_name, file_date)

Finished Processing 20220401.json


In [8]:
file_date

datetime.date(2022, 4, 1)

In [77]:
result = None
for file_name in list_file:
    date_str = file_name.split('_')[-1].split('.')[0]
    file_date = datetime.strptime(date_str, "%Y%m%d").date()
    
    start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
    end_date = datetime.strptime(end_date_str, "%Y%m%d").date()
    if start_date <= file_date <= end_date:
        df = etl_process(path, file_name, file_date)
        if result is None:
            result = df
        else:
            result = result.union(df)
            
result_group = result.groupby('Contract','Type','Date').sum()
result_group = result_group.withColumnRenamed('sum(TotalDuration)','TotalDuration')
final = result_group.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
final = final.withColumnRenamed('Giải Trí', 'RelaxDuration') \
    .withColumnRenamed('Phim Truyện', 'MovieDuration') \
    .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
    .withColumnRenamed('Thể Thao', 'SportDuration') \
    .withColumnRenamed('Truyền Hình', 'TVDuration')

Finished Processing 20220401.json
Finished Processing 20220402.json
Finished Processing 20220403.json
Finished Processing 20220404.json
Finished Processing 20220405.json
Finished Processing 20220406.json
Finished Processing 20220407.json
Finished Processing 20220408.json
Finished Processing 20220409.json
Finished Processing 20220410.json
Finished Processing 20220411.json
Finished Processing 20220412.json
Finished Processing 20220413.json
Finished Processing 20220414.json
Finished Processing 20220415.json
Finished Processing 20220416.json
Finished Processing 20220417.json
Finished Processing 20220418.json
Finished Processing 20220419.json
Finished Processing 20220420.json
Finished Processing 20220421.json
Finished Processing 20220422.json
Finished Processing 20220423.json
Finished Processing 20220424.json
Finished Processing 20220425.json
Finished Processing 20220426.json
Finished Processing 20220427.json
Finished Processing 20220428.json
Finished Processing 20220429.json
Finished Proce

In [33]:
final.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- RelaxDuration: long (nullable = true)
 |-- MovieDuration: long (nullable = true)
 |-- ChildDuration: long (nullable = true)
 |-- SportDuration: long (nullable = true)
 |-- TVDuration: long (nullable = true)



calculate most_watch

In [37]:
result_group.show(10)

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|DNH014998|Phim Truyện|       112733|
|HND486882|Phim Truyện|        67217|
|HUFD07189|Truyền Hình|        49687|
|HDFD36288|Truyền Hình|       184878|
|CTFD04401|Truyền Hình|      1398087|
|HNH954607|Phim Truyện|        15972|
|HNH855959|Truyền Hình|       171875|
|SGH034683|Truyền Hình|       623300|
|NTFD35330|Truyền Hình|       179719|
|NTFD48198|Phim Truyện|       187513|
+---------+-----------+-------------+
only showing top 10 rows



In [20]:
windowSpec = Window.partitionBy("Contract").orderBy(desc("TotalDuration"))
mostWatch = result_group.withColumn("rank",row_number().over(windowSpec))
mostWatch = mostWatch.filter(mostWatch.rank==1)
mostWatch = mostWatch.select("Contract", "Type")
mostWatch = mostWatch.withColumnRenamed("Type","MostWatch")
mostWatch.show()

+--------------+-----------+
|      Contract|  MostWatch|
+--------------+-----------+
|113.182.209.48|   Giải Trí|
|14.182.110.125|Truyền Hình|
|     AGAAA0338|Truyền Hình|
|     AGAAA0342|Truyền Hình|
|     AGAAA0346|Truyền Hình|
|     AGAAA0353|Truyền Hình|
|     AGAAA0391|Truyền Hình|
|     AGAAA0452|Truyền Hình|
|     AGAAA0550|Truyền Hình|
|     AGAAA0593|Truyền Hình|
|     AGAAA0613|Truyền Hình|
|     AGAAA0638|Truyền Hình|
|     AGAAA0663|Truyền Hình|
|     AGAAA0692|Truyền Hình|
|     AGAAA0693|Truyền Hình|
|     AGAAA0718|Truyền Hình|
|     AGAAA0723|Truyền Hình|
|     AGAAA0729|Truyền Hình|
|     AGAAA0732|Truyền Hình|
|     AGAAA0737|Truyền Hình|
+--------------+-----------+
only showing top 20 rows



In [38]:
def most_watch_calc(result):
    windowSpec = Window.partitionBy("Contract").orderBy(desc("TotalDuration"))
    most_watch = result_group.withColumn("rank",row_number().over(windowSpec))
    most_watch = most_watch.filter(most_watch.rank==1)
    most_watch = most_watch.select("Contract", "Type")
    most_watch = most_watch.withColumnRenamed("Type","MostWatch")
    return most_watch

calculate customer taste

In [24]:
final.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Date: date (nullable = false)
 |-- RelaxDuration: long (nullable = true)
 |-- MovieDuration: long (nullable = true)
 |-- ChildDuration: long (nullable = true)
 |-- SportDuration: long (nullable = true)
 |-- TVDuration: long (nullable = true)



In [39]:
final_test = final.withColumn("RelaxDuration",when(col("RelaxDuration").isNotNull(),"Relax").otherwise(col("RelaxDuration")))
final_test = final_test.withColumn("MovieDuration",when(col("MovieDuration").isNotNull(),"Movie").otherwise(col("MovieDuration")))
final_test = final_test.withColumn("ChildDuration",when(col("ChildDuration").isNotNull(),"Child").otherwise(col("ChildDuration")))
final_test = final_test.withColumn("SportDuration",when(col("SportDuration").isNotNull(),"Sport").otherwise(col("SportDuration")))
final_test = final_test.withColumn("TVDuration",when(col("TVDuration").isNotNull(),"TV").otherwise(col("TVDuration")))

In [40]:
final_test.show()

+---------+-------------+-------------+-------------+-------------+----------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+-------------+-------------+-------------+-------------+----------+
|HTFD11598|         null|        Movie|         null|         null|        TV|
|HPFD48556|        Relax|         null|         null|         null|        TV|
|NBFD10014|         null|         null|         null|         null|        TV|
|HNH619088|         null|        Movie|        Child|         null|        TV|
|HNH036174|         null|        Movie|         null|         null|        TV|
|DNH067877|         null|         null|         null|         null|        TV|
|SGH806190|         null|         null|         null|         null|        TV|
|HNH582022|         null|         null|         null|         null|        TV|
|HNH795510|         null|        Movie|        Child|         null|        TV|
|DNFD91557|         null|         null|         null

In [41]:
def customer_taste_calc(final):
    final = final.withColumn("RelaxDuration",when(col("RelaxDuration").isNotNull(),"Relax").otherwise(col("RelaxDuration")))
    final = final.withColumn("MovieDuration",when(col("MovieDuration").isNotNull(),"Movie").otherwise(col("MovieDuration")))
    final = final.withColumn("ChildDuration",when(col("ChildDuration").isNotNull(),"Child").otherwise(col("ChildDuration")))
    final = final.withColumn("SportDuration",when(col("SportDuration").isNotNull(),"Sport").otherwise(col("SportDuration")))
    final = final.withColumn("TVDuration",when(col("TVDuration").isNotNull(),"TV").otherwise(col("TVDuration")))
    
    taste = final.withColumn('CustomerTaste', sf.concat_ws("-", *[i for i in final.columns if i != ['Contract']]))
    taste = taste.select('Contract','CustomerTaste')
    return taste

In [78]:
def calculate_customer_taste(df):
    for column in df.columns:
        if 'Duration' in column:
            df = df.withColumn(column, when(col(column).isNotNull(), column.replace("Duration", "")))
    customer_taste = df.withColumn('CustomerTaste',
                                   sf.concat_ws("-", *[i for i in df.columns if 'Duration' in i]))
    customer_taste = customer_taste.select('Contract', 'CustomerTaste')
    return customer_taste

customer_taste = calculate_customer_taste(final)
customer_taste.show(10, truncate=False)

23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:04:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+-------------+
|Contract |CustomerTaste|
+---------+-------------+
|LDFD43130|Movie-TV     |
|DLD010292|Movie-TV     |
|YBFD04352|Movie-TV     |
|NBFD20238|Movie-TV     |
|HNH371198|Child-TV     |
|QNFD15570|TV           |
|AGFD24376|TV           |
|HNJ165212|Child-TV     |
|DAFD06623|Movie-TV     |
|HNH889692|TV           |
+---------+-------------+
only showing top 10 rows



In [80]:
final.show()

23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 01:08:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+----------+-------------+-------------+-------------+-------------+----------+
| Contract|      Date|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+----------+-------------+-------------+-------------+-------------+----------+
|LDFD43130|2022-04-01|         null|         7256|         null|         null|     11999|
|DLD010292|2022-04-01|         null|          901|         null|         null|     10244|
|YBFD04352|2022-04-01|         null|           51|         null|         null|      2388|
|NBFD20238|2022-04-01|         null|          593|         null|         null|       483|
|HNH371198|2022-04-01|         null|         null|        15735|         null|     40395|
|QNFD15570|2022-04-01|         null|         null|         null|         null|      4982|
|AGFD24376|2022-04-01|         null|         null|         null|         null|      3143|
|HNJ165212|2022-04-01|         null|         null|          299|         null|        24|
|DAFD06623

calculate activeness

In [42]:
(end_date-start_date).days+1

30

In [53]:
active = result.groupby('Contract','Type','Date').sum()
active = active.withColumnRenamed('sum(TotalDuration)','TotalDuration')
active = active.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
active = active.withColumnRenamed('Giải Trí', 'RelaxDuration') \
    .withColumnRenamed('Phim Truyện', 'MovieDuration') \
    .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
    .withColumnRenamed('Thể Thao', 'SportDuration') \
    .withColumnRenamed('Truyền Hình', 'TVDuration')

In [54]:
active.show()

23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+----------+-------------+-------------+-------------+-------------+----------+
| Contract|      Date|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+----------+-------------+-------------+-------------+-------------+----------+
|LDFD43130|2022-04-01|         null|         7256|         null|         null|     11999|
|DLD010292|2022-04-01|         null|          901|         null|         null|     10244|
|YBFD04352|2022-04-01|         null|           51|         null|         null|      2388|
|NBFD20238|2022-04-01|         null|          593|         null|         null|       483|
|HNH371198|2022-04-01|         null|         null|        15735|         null|     40395|
|QNFD15570|2022-04-01|         null|         null|         null|         null|      4982|
|AGFD24376|2022-04-01|         null|         null|         null|         null|      3143|
|HNJ165212|2022-04-01|         null|         null|          299|         null|        24|
|DAFD06623

In [55]:
active.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Date: date (nullable = false)
 |-- RelaxDuration: long (nullable = true)
 |-- MovieDuration: long (nullable = true)
 |-- ChildDuration: long (nullable = true)
 |-- SportDuration: long (nullable = true)
 |-- TVDuration: long (nullable = true)



In [57]:
active = active.filter((active.Date >= start_date) & (active.Date <= end_date))
active = active.withColumn("IsActive",
                          sf.when((active.RelaxDuration > 0) | (active.MovieDuration > 0) | (active.ChildDuration > 0) | (active.SportDuration > 0) | (active.TVDuration > 0), 1)
                          .otherwise(0))

In [58]:
# Calculate the total active dates for each user
total_days = (end_date-start_date).days+1
activeness = active.groupBy("Contract").agg(sf.sum("IsActive").alias("ActiveDays"))

# Calculate the active rate for each user
activeness = activeness.withColumn("ActiveRate", (sf.col("ActiveDays") / sf.lit(total_days)).cast("double"))
activeness.show(10)

23/11/08 00:08:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:08:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+----------+------------------+
| Contract|ActiveDays|        ActiveRate|
+---------+----------+------------------+
|HYFD55120|        30|               1.0|
|HNJ132019|        30|               1.0|
|SGH498259|        29|0.9666666666666667|
|QAFD12708|        30|               1.0|
|BTFD13949|        26|0.8666666666666667|
|GLD010540|        30|               1.0|
|SGD480362|        30|               1.0|
|NDFD03091|        23|0.7666666666666667|
|LDFD21316|        30|               1.0|
|DNH034003|        30|               1.0|
+---------+----------+------------------+
only showing top 10 rows



In [59]:
activeness.filter(col('Contract')=='DAFD98771').show() #should be 14 days

+---------+----------+------------------+
| Contract|ActiveDays|        ActiveRate|
+---------+----------+------------------+
|DAFD98771|        14|0.4666666666666667|
+---------+----------+------------------+



In [64]:
start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
end_date = datetime.strptime(end_date_str, "%Y%m%d").date()

def calculate_active_rate(result):
    active = result.groupby('Contract','Type','Date').sum()
    active = active.withColumnRenamed('sum(TotalDuration)','TotalDuration')
    active = active.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
    active = active.withColumnRenamed('Giải Trí', 'RelaxDuration') \
                    .withColumnRenamed('Phim Truyện', 'MovieDuration') \
                    .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
                    .withColumnRenamed('Thể Thao', 'SportDuration') \
                    .withColumnRenamed('Truyền Hình', 'TVDuration')

    active = active.filter((active.Date >= start_date) & (active.Date <= end_date))
    active = active.withColumn("IsActive",
                          sf.when((active.RelaxDuration > 0) | (active.MovieDuration > 0) | (active.ChildDuration > 0) | (active.SportDuration > 0) | (active.TVDuration > 0), 1)
                          .otherwise(0))
    
    # Calculate the total active dates for each user
    total_days = (end_date-start_date).days+1
    activeness = active.groupBy("Contract").agg(sf.sum("IsActive").alias("ActiveDays"))

    # Calculate the active rate for each user
    activeness = activeness.withColumn("ActiveRate", (sf.col("ActiveDays") / sf.lit(total_days)).cast("double"))
    
    return activeness

In [73]:
start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
end_date = datetime.strptime(end_date_str, "%Y%m%d").date()

def calculate_activeness(df):
    total_days = (end_date - start_date).days + 1
    
    active = df.groupby('Contract', 'Date').agg((sf.sum('TotalDuration').alias('TotalDurationPerDay')))
    active = active.withColumn("IsActive", sf.when(active.TotalDurationPerDay > 0, 1).otherwise(0))

    activeness = active.groupBy("Contract").agg(sf.sum("IsActive").alias("ActiveDays"))
    activeness = activeness.withColumn("ActiveRate", (sf.col("ActiveDays") / sf.lit(total_days)))
    return activeness


activeness = calculate_activeness(result)
activeness.show(10, truncate=False)

+---------+----------+------------------+
|Contract |ActiveDays|ActiveRate        |
+---------+----------+------------------+
|QNFD34621|30        |1.0               |
|SGH494322|30        |1.0               |
|HYFD47097|30        |1.0               |
|HYFD55120|30        |1.0               |
|HUFD05523|30        |1.0               |
|NDFD19838|30        |1.0               |
|SGJ005892|30        |1.0               |
|DAFD71727|22        |0.7333333333333333|
|HPFD48943|30        |1.0               |
|QNFD77422|30        |1.0               |
+---------+----------+------------------+
only showing top 10 rows



In [74]:
activeness.filter(col('Contract')=='DAFD98771').show() #should be 14 days

+---------+----------+------------------+
| Contract|ActiveDays|        ActiveRate|
+---------+----------+------------------+
|DAFD98771|        14|0.4666666666666667|
+---------+----------+------------------+



combine most_watch, tase, activeness, final

In [61]:
most_watch = most_watch_calc(result)

In [62]:
customer_taste = customer_taste_calc(final)
customer_taste.show(10, truncate=False)

+---------+------------------------+
|Contract |CustomerTaste           |
+---------+------------------------+
|HTFD11598|HTFD11598-Movie-TV      |
|HPFD48556|HPFD48556-Relax-TV      |
|NBFD10014|NBFD10014-TV            |
|HNH619088|HNH619088-Movie-Child-TV|
|HNH036174|HNH036174-Movie-TV      |
|DNH067877|DNH067877-TV            |
|SGH806190|SGH806190-TV            |
|HNH582022|HNH582022-TV            |
|HNH795510|HNH795510-Movie-Child-TV|
|DNFD91557|DNFD91557-TV            |
+---------+------------------------+
only showing top 10 rows



In [66]:
activeness = calculate_active_rate(result)
activeness.show(10, truncate=False)

23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:17:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+----------+------------------+
|Contract |ActiveDays|ActiveRate        |
+---------+----------+------------------+
|HYFD55120|30        |1.0               |
|HNJ132019|30        |1.0               |
|SGH498259|29        |0.9666666666666667|
|QAFD12708|30        |1.0               |
|BTFD13949|26        |0.8666666666666667|
|GLD010540|30        |1.0               |
|SGD480362|30        |1.0               |
|NDFD03091|23        |0.7666666666666667|
|LDFD21316|30        |1.0               |
|DNH034003|30        |1.0               |
+---------+----------+------------------+
only showing top 10 rows



In [67]:
# 1st join with most_watch
final_test = final.join(most_watch, on='Contract', how='left')

# 2nd join with customer_taste
final_test = final_test.join(customer_taste, on='Contract', how='left')

# 3rd join with activeness
final_test = final_test.join(activeness, on='Contract', how='left')

final_test.show()


23/11/08 00:21:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:21:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:22:16 WARN DAGScheduler: Broadcasting large task bin

+---------+-------------+-------------+-------------+-------------+----------+-----------+--------------------+----------+------------------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|  MostWatch|       CustomerTaste|ActiveDays|        ActiveRate|
+---------+-------------+-------------+-------------+-------------+----------+-----------+--------------------+----------+------------------+
|DLFD14250|         null|        29144|         null|         null|    391492|Truyền Hình|  DLFD14250-Movie-TV|        22|0.7333333333333333|
|DNFD91557|         null|         null|         null|         null|     95567|Truyền Hình|        DNFD91557-TV|        27|               0.9|
|DNH034235|         null|         null|         null|         null|    139582|Truyền Hình|        DNH034235-TV|        30|               1.0|
|DNH067877|         null|         null|         null|         null|    181308|Truyền Hình|        DNH067877-TV|        29|0.9666666666666667|
|DNH08

In [48]:
customer_taste.filter(col('Contract')=='CTD037046').show()

+---------+--------------+
| Contract| CustomerTaste|
+---------+--------------+
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|Relax-Child-TV|
|CTD037046|            TV|
|CTD037046|      Child-TV|
|CTD037046|            TV|
|CTD037046|      Child-TV|
|CTD037046|            TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|            TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
|CTD037046|      Child-TV|
+---------+--------------+
only showing top 20 rows



In [68]:
def join_dataframes(final, most_watch, customer_taste, activeness):
    final_df = final.join(most_watch, on='Contract', how='left')
    final_df = final_df.join(customer_taste, on='Contract', how='left')
    final_df = final_df.join(activeness, on='Contract', how='left')
    return final_df

final_df = join_dataframes(final, most_watch, customer_taste, activeness)
final_df.show()

23/11/08 00:47:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:47:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 00:48:02 WARN DAGScheduler: Broadcasting large task bin

+---------+-------------+-------------+-------------+-------------+----------+-----------+--------------------+----------+------------------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|  MostWatch|       CustomerTaste|ActiveDays|        ActiveRate|
+---------+-------------+-------------+-------------+-------------+----------+-----------+--------------------+----------+------------------+
|DLFD14250|         null|        29144|         null|         null|    391492|Truyền Hình|  DLFD14250-Movie-TV|        22|0.7333333333333333|
|DNFD91557|         null|         null|         null|         null|     95567|Truyền Hình|        DNFD91557-TV|        27|               0.9|
|DNH034235|         null|         null|         null|         null|    139582|Truyền Hình|        DNH034235-TV|        30|               1.0|
|DNH067877|         null|         null|         null|         null|    181308|Truyền Hình|        DNH067877-TV|        29|0.9666666666666667|
|DNH08